In [2]:
import sys
sys.path.append('/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero')

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import argparse
from transformers import Qwen2VLForConditionalGeneration, Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import json
import pdb

import cv2
from PIL import Image as PILImage
import re
from sam2.sam2_image_predictor import SAM2ImagePredictor
import numpy as np
import matplotlib.pyplot as plt

/home/ksmehrab/miniconda/envs/segzero/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# configuration object, to replace args
class Config:
    def __init__(self):
        self.reasoning_model_path = "/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero/pretrained_models/VisionReasoner-7B"
        self.segmentation_model_path = "facebook/sam2-hiera-large"

args = Config()

In [5]:
def extract_json_bbox_points_think(output_text, x_factor, y_factor):
    json_match = re.search(r'<answer>\s*(.*?)\s*</answer>', output_text, re.DOTALL)
    json_string = json_match.group(1)
    if json_match:
        data = json.loads(json_string)
        pred_bboxes = [[
            int(item['bbox_2d'][0] * x_factor + 0.5),
            int(item['bbox_2d'][1] * y_factor + 0.5),
            int(item['bbox_2d'][2] * x_factor + 0.5),
            int(item['bbox_2d'][3] * y_factor + 0.5)
        ] for item in data]
        pred_points = [[
            int(item['point_2d'][0] * x_factor + 0.5),
            int(item['point_2d'][1] * y_factor + 0.5)
        ] for item in data]
    
    think_pattern = r'<think>([^<]+)</think>'
    think_match = re.search(think_pattern, output_text)
    think_text = ""
    if think_match:
        think_text = think_match.group(1)
    
    return json_string, pred_bboxes, pred_points, think_text

In [6]:
def extract_bbox_points_think(output_text, x_factor, y_factor):
    json_match = re.search(r'<answer>\s*(.*?)\s*</answer>', output_text, re.DOTALL)
    if json_match:
        data = json.loads(json_match.group(1))
        pred_bboxes = [[
            int(item['bbox_2d'][0] * x_factor + 0.5),
            int(item['bbox_2d'][1] * y_factor + 0.5),
            int(item['bbox_2d'][2] * x_factor + 0.5),
            int(item['bbox_2d'][3] * y_factor + 0.5)
        ] for item in data]
        pred_points = [[
            int(item['point_2d'][0] * x_factor + 0.5),
            int(item['point_2d'][1] * y_factor + 0.5)
        ] for item in data]
    
    think_pattern = r'<think>([^<]+)</think>'
    think_match = re.search(think_pattern, output_text)
    think_text = ""
    if think_match:
        think_text = think_match.group(1)
    
    return pred_bboxes, pred_points, think_text

In [7]:
def normalize_bbox(bbox, x_factor, y_factor):
    x1 = int(bbox[0] * x_factor + 0.5)
    y1 = int(bbox[1] * y_factor + 0.5)
    x2 = int(bbox[2] * x_factor + 0.5)
    y2 = int(bbox[3] * y_factor + 0.5)

    return [x1, y1, x2, y2]

In [8]:
def normalize_point(point, x_factor, y_factor):
    x = int(point[0] * x_factor + 0.5)
    y = int(point[1] * y_factor + 0.5)

    return [x, y]

In [9]:
def safe_json_array_loads(json_string):
    """
    Safely load a JSON string which contains a list of dictionaries. If it fails, return an empty list.

    Args:
        json_string (str): The JSON string to be parsed.
    Returns:
        list: The parsed JSON object, or an empty list if parsing fails. 
    """
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        return []

In [10]:
def all_inner_lists_empty(list_of_lists):
    """
    Check if all inner lists in a list of lists are empty.
    
    Args:
        list_of_lists: A list containing other lists
        
    Returns:
        bool: True if all inner lists are empty, False otherwise
    """
    # Check if the outer list itself is empty
    if not list_of_lists:
        return True
    
    # Check if all inner lists are empty
    return all(len(inner_list) == 0 for inner_list in list_of_lists)

In [ ]:
# def extract_clean_json_string(text):
#     """
#     Extract clean JSON string from text that may contain markdown code blocks or other formatting.
    
#     Args:
#         text (str): Input text containing JSON
        
#     Returns:
#         str: Clean JSON string, or empty string if no valid JSON found
#     """
#     import re
#     import json
    
#     # First, try to find JSON within markdown code blocks
#     markdown_pattern = r'```(?:json)?\s*(.*?)\s*```'
#     markdown_match = re.search(markdown_pattern, text, re.DOTALL)
    
#     if markdown_match:
#         json_candidate = markdown_match.group(1).strip()
#     else:
#         # If no markdown blocks, look for JSON-like structures
#         # Look for content between [ ] or { }
#         bracket_pattern = r'(\[.*?\]|\{.*?\})'
#         bracket_match = re.search(bracket_pattern, text, re.DOTALL)
#         if bracket_match:
#             json_candidate = bracket_match.group(1).strip()
#         else:
#             json_candidate = text.strip()
    
#     # Clean up common formatting issues
#     json_candidate = json_candidate.replace('\t', ' ')  # Replace tabs with spaces
#     json_candidate = re.sub(r'\s+', ' ', json_candidate)  # Normalize whitespace

#     print(json_candidate)
    
#     # Validate that it's proper JSON
#     try:
#         json.loads(json_candidate)
#         return json_candidate
#     except json.JSONDecodeError:
#         print("Invalid JSON detected.")
#         return ""

In [69]:
def extract_clean_json_string(text):
    """
    Extract clean JSON string from text that may contain markdown code blocks or other formatting.
    
    Args:
        text (str): Input text containing JSON
        
    Returns:
        str: Clean JSON string, or empty string if no valid JSON found
    """
    
    # First, try to find JSON within markdown code blocks
    markdown_pattern = r'```(?:json)?\s*(.*?)\s*```'
    markdown_match = re.search(markdown_pattern, text, re.DOTALL)
    
    if markdown_match:
        json_candidate = markdown_match.group(1).strip()
    else:
        # Function to find matching bracket
        def find_matching_bracket(text, start_pos, open_char, close_char):
            count = 1
            pos = start_pos + 1
            while pos < len(text) and count > 0:
                if text[pos] == open_char:
                    count += 1
                elif text[pos] == close_char:
                    count -= 1
                pos += 1
            return pos if count == 0 else -1
        
        # Look for JSON array or object
        json_candidate = ""
        
        # First try to find array starting with [
        array_start = text.find('[')
        if array_start != -1:
            array_end = find_matching_bracket(text, array_start, '[', ']')
            if array_end != -1:
                json_candidate = text[array_start:array_end]
        
        # If no valid array found, try to find object starting with {
        if not json_candidate:
            obj_start = text.find('{')
            if obj_start != -1:
                obj_end = find_matching_bracket(text, obj_start, '{', '}')
                if obj_end != -1:
                    json_candidate = text[obj_start:obj_end]
        
        # If still no candidate, fallback to original approach
        if not json_candidate:
            json_candidate = text.strip()
    
    # Clean up common formatting issues
    json_candidate = json_candidate.replace('\t', ' ')  # Replace tabs with spaces
    json_candidate = re.sub(r'\s+', ' ', json_candidate)  # Normalize whitespace

    # print(json_candidate)
    
    # Validate that it's proper JSON
    try:
        json.loads(json_candidate)
        return json_candidate
    except json.JSONDecodeError:
        print("Invalid JSON detected.")
        return ""

In [70]:
def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 0
    return intersection, union

In [71]:
def compute_bbox_iou(bbox1, bbox2):
    """
    Compute IoU (Intersection over Union) between two bounding boxes.
    
    Args:
        bbox1: List or array [x1, y1, x2, y2] representing first bbox
        bbox2: List or array [x1, y1, x2, y2] representing second bbox
        
    Returns:
        float: IoU value between 0 and 1
    """

    x1_1, y1_1, x2_1, y2_1 = bbox1
    x1_2, y1_2, x2_2, y2_2 = bbox2
    
    # Calculate intersection coordinates
    x1_intersect = max(x1_1, x1_2)
    y1_intersect = max(y1_1, y1_2)
    x2_intersect = min(x2_1, x2_2)
    y2_intersect = min(y2_1, y2_2)
    
    # Check if there's no intersection
    if x1_intersect >= x2_intersect or y1_intersect >= y2_intersect:
        return 0.0
    
    # Calculate intersection area
    intersection_area = (x2_intersect - x1_intersect) * (y2_intersect - y1_intersect)
    
    # Calculate areas of both bboxes
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate union area
    union_area = area1 + area2 - intersection_area
    
    # Avoid division by zero
    if union_area == 0:
        return 0.0
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou

In [50]:
def combine_masks(masks):
    combined_mask = masks[0]
    for i in range(1, len(masks)):
        combined_mask = combined_mask | masks[i]
    return combined_mask

In [15]:
#We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
reasoning_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    args.reasoning_model_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


In [ ]:
reasoning_model.eval()

In [17]:
segmentation_model = SAM2ImagePredictor.from_pretrained(args.segmentation_model_path)

In [18]:
# default processer
processor = AutoProcessor.from_pretrained(args.reasoning_model_path, padding_side="left")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [20]:
QUESTION_TEMPLATE = \
    "Please find \"{Question}\" with bboxs and points." \
    "Compare the difference between object(s) or parts and find the most closely matched object(s) or parts." \
    "Output the thinking process in <think> </think> and final answer in <answer> </answer> tags." \
    "Output the bbox(es) and point(s) inside the interested object(s) or parts in JSON format." \
    "i.e., <think> thinking process here </think>" \
    "<answer>{Answer}</answer>"

In [21]:
# The box caption template will be used to caption the first predicted region/s 
BOX_CAPTION_TEMPLATE = "Please label the regions inside the provided bboxes. Be concise. Output the same JSON as provided, adding a \"label\" field per region. Regions to label: \n <JSON_BBOX_INFO>"

# BOX_CAPTION_TEMPLATE = "Please label the regions inside the provided bboxes. Be concise. Output the same JSON as provided, adding a \"label\" field per region. Do not generate any new bboxes. Only label the regions in the provided bboxes. Regions to label: \n <JSON_BBOX_INFO>"

In [72]:
# Self critique template, similar to QUESTION_TEMPLATE but adds the previous answer for self-evaluation 
# TODO: Should we change {Answer} to the previous answer? 
# TODO: Should we extract the caption from the previous answer and provide it as a separate <CAPTION_INFO>?

# SELF_CRITIQUE_TEMPLATE = \
#     "The following bbox(es) and captions are provided for your reference as a previous answer: <JSON_BBOX_INFO>." \
#     "Please find \"{Question}\" with bboxs and points." \
#     "Based on the reference bbox(es) and point(s), decide if the previous answer was correct or not." \
#     "Think if the predicted bbox(es) need to be bigger, smaller or the same to accurately find \"{Question}\"" \
#     "Output the thinking process in <think> </think> and final answer in <answer> </answer> tags." \
#     "Output the bbox(es) and point(s) inside the interested object(s) or parts in JSON format." \
#     "i.e., <think> thinking process here </think>" \
#     "<answer>{Answer}</answer>"

SELF_CRITIQUE_TEMPLATE = \
    "Please find \"{Question}\" with bbox(es) and points. " \
    "The following bbox(es) were your previous answer to find \"{Question}\": <JSON_BBOX_INFO>. " \
    "Here are the captions of you what you actually found in the previous bbox(es): <JSON_CAPTION_INFO> " \
    "Compare what you need to find --  \"{Question}\" with the captioned bboxes to decide if the previous answer was correct or not. " \
    "The bbox(es) should only enclose \"{Question}\" for it to be correct." \
    "For example, if your task is to find \"person's head\", but the caption of the bbox(es) say that it is the entire person, then the previous answer is incorrect. " \
    "If the previous bbox(es) was incorrect, think if the correct bbox(es) need to be bigger, smaller or the same to accurately find \"{Question}\". " \
    "Output the thinking process in <think> </think> and final answer in <answer> </answer> tags. " \
    "Output the bbox(es) and point(s) inside the interested object(s) or parts in JSON format. " \
    "i.e., <think> thinking process here </think> " \
    "<answer>{Answer}</answer>"



In [73]:
def generate_outputs_for_self_critique_template(
    filename, 
    seg_list,
    json_bboxes_strings, 
    image, 
    resize_size=840
):
    """ 
    Pass 3
    """ 
    assert len(seg_list) == len(json_bboxes_strings)
    original_width, original_height = image.size
    messages = []
    for prev_output, seg_name in zip(json_bboxes_strings, seg_list):
        # separate the previous output into bbox and caption
        prev_output_json_string = extract_clean_json_string(prev_output)
        prev_output_json = safe_json_array_loads(prev_output_json_string)
        prev_bboxes = [item['bbox_2d'] for item in prev_output_json if 'bbox_2d' in item]
        prev_captions = [item['label'] for item in prev_output_json if 'label' in item]

        # convert the separate prev_bboxes and prev_captions into a JSON string
        prev_bboxes_json_string = json.dumps([{'bbox_2d': bbox} for bbox in prev_bboxes])
        prev_captions_json_string = json.dumps([{'caption': caption} for caption in prev_captions])

        formatted_template = SELF_CRITIQUE_TEMPLATE.format(
            Question=seg_name.lower().strip("."),
            Answer="[{\"bbox_2d\": [10,100,200,210], \"point_2d\": [30,110]}, {\"bbox_2d\": [225,296,706,786], \"point_2d\": [302,410]}]"
        )
        final_text = formatted_template.replace(
            "<JSON_BBOX_INFO>", prev_bboxes_json_string).replace(
                "<JSON_CAPTION_INFO>", prev_captions_json_string)
    
        message = [{
            "role": "user",
            "content": [
            {
                "type": "image", 
                "image": image.resize((resize_size, resize_size), PILImage.BILINEAR)
            },
            {   
                "type": "text",
                "text": final_text
            }
        ]
        }]
        messages.append(message)
    
    # print(messages)
    
    # Preparation for inference
    text = [processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) for msg in messages]

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=text,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        generated_ids = reasoning_model.generate(**inputs, use_cache=True, max_new_tokens=1024, do_sample=False)

        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        batch_output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        # print(batch_output_text)
        batch_json_strings = []
        batch_bboxes = []
        batch_points = []
        batch_thinks = []

        for id_idx in range(len(batch_output_text)):
            try:
                json_string, bboxes, points, think = extract_json_bbox_points_think(
                                        batch_output_text[id_idx], 
                                        original_width/resize_size, 
                                        original_height/resize_size
                                    )
            except Exception as e:
                # assign empty values if there is an exception
                print("Reasoning error: ", e, "Text: ", batch_output_text[id_idx], "Image: ", filename, "Seg: ", seg_list[id_idx])
                think = ""
                bboxes = []
                points = []
                json_string = ""

            batch_json_strings.append(json_string)
            batch_bboxes.append(bboxes)
            batch_points.append(points)
            batch_thinks.append(think)

    # clean GPU memory
    del inputs, generated_ids, generated_ids_trimmed
    torch.cuda.empty_cache()

    return batch_json_strings, batch_bboxes, batch_points, batch_thinks
    

In [74]:
def generate_outputs_for_caption_template(
    filename, 
    json_bboxes_strings, 
    image, 
    resize_size=840
):
    """ 
    Pass 2
    """
    # TODO: handle situation where an element in the json_bboxes_strings is empty. 
    # This can happen if the first pass detecting bbox does not detect anything.
    # Or if the first pass runs into an exception 

    original_width, original_height = image.size
    messages = []
    for args_text in json_bboxes_strings:
        message = [{
            "role": "user",
            "content": [
            {
                "type": "image", 
                "image": image.resize((resize_size, resize_size), PILImage.BILINEAR)
            },
            {   
                "type": "text",
                "text": BOX_CAPTION_TEMPLATE.replace("<JSON_BBOX_INFO>", args_text)
            }
        ]
        }]
        messages.append(message)
    
    # print(messages)
    
    # Preparation for inference
    text = [processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) for msg in messages]

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=text,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        generated_ids = reasoning_model.generate(**inputs, use_cache=True, max_new_tokens=1024, do_sample=False)

        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        batch_output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

    output_texts_cleaned = []
    for output_text in batch_output_text:
        output_texts_cleaned.append(extract_clean_json_string(output_text))
        
    # clean GPU memory
    del inputs, generated_ids, generated_ids_trimmed
    torch.cuda.empty_cache()

    return output_texts_cleaned
    

In [75]:
def generate_outputs_for_question_template(
    filename,
    seg_list,
    image,
    resize_size=840,
):
    """ 
    Pass 1 
    """
    original_width, original_height = image.size
    messages = []
    for args_text in seg_list:
        message = [{
            "role": "user",
            "content": [
            {
                "type": "image", 
                "image": image.resize((resize_size, resize_size), PILImage.BILINEAR)
            },
            {   
                "type": "text",
                "text": QUESTION_TEMPLATE.format(
                    Question=args_text.lower().strip("."),
                    Answer="[{\"bbox_2d\": [10,100,200,210], \"point_2d\": [30,110]}, {\"bbox_2d\": [225,296,706,786], \"point_2d\": [302,410]}]"
                )    
            }
        ]
        }]
        messages.append(message)
    
    # Preparation for inference
    text = [processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) for msg in messages]

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=text,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output

    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
        generated_ids = reasoning_model.generate(**inputs, use_cache=True, max_new_tokens=1024, do_sample=False)

        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        batch_output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=False, clean_up_tokenization_spaces=False
        )

        # print(batch_output_text)
        batch_json_strings = []
        batch_bboxes = []
        batch_points = []
        batch_thinks = []

        for id_idx in range(len(batch_output_text)):
            try:
                json_string, bboxes, points, think = extract_json_bbox_points_think(
                                        batch_output_text[id_idx], 
                                        original_width/resize_size, 
                                        original_height/resize_size
                                    )
            except Exception as e:
                # assign empty values if there is an exception
                print("Reasoning error: ", e, "Text: ", batch_output_text[id_idx], "Image: ", filename, "Seg: ", seg_list[id_idx])
                think = ""
                bboxes = []
                points = []
                json_string = ""

            batch_json_strings.append(json_string)
            batch_bboxes.append(bboxes)
            batch_points.append(points)
            batch_thinks.append(think)

    # clean GPU memory
    del inputs, generated_ids, generated_ids_trimmed
    torch.cuda.empty_cache()

    return batch_json_strings, batch_bboxes, batch_points, batch_thinks

In [79]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
from PIL import Image as PILImage

def visualize_bbox_comparison(
        image, 
        pass1_json_strings, 
        pass2_json_strings_cleaned, 
        pass3_input_json_strings,
        pass3_output_bboxes, 
        pass3_predicted_masks, 
        gt_masks,
        pass3_output_thinks,
        seg_list, 
        filename,
        x_factor, 
        y_factor
):
    """
    Create comprehensive visualization comparing Pass 1 and Pass 2 results
    
    Args:
        image: PIL Image object
        pass1_json_strings: List of JSON strings from Pass 1 (QUESTION_TEMPLATE)
        pass2_json_strings_cleaned: List of cleaned JSON strings from Pass 2 (BOX_CAPTION_TEMPLATE)
        pass3_input_json_strings: List of JSON strings used as input for Pass 3 (SELF_CRITIQUE_TEMPLATE)
        seg_list: List of object/part names
        filename: Image filename for saving plots
    """

    for i, (pass1_json_str, pass2_json_str, pass3_input_json_str, pass3_output_bbox, pass3_predicted_mask, gt_mask, pass3_output_think, seg_name) in enumerate(zip(pass1_json_strings, pass2_json_strings_cleaned, pass3_input_json_strings, pass3_output_bboxes, pass3_predicted_masks, gt_masks, pass3_output_thinks, seg_list)):
        # Skip if pass1_json_str is empty (no detection in Pass 1)
        if not pass1_json_str:
            print(f"Skipping {seg_name} - no detection in Pass 1")
            continue
            
        try:
            # Parse Pass 1 results (initial bboxes)
            pass1_data = json.loads(pass1_json_str)
            pass1_bboxes = [normalize_bbox(item['bbox_2d'], x_factor, y_factor) for item in pass1_data]
            
            # Parse Pass 2 results (captioned bboxes)
            if pass2_json_str:
                pass2_data = json.loads(pass2_json_str)
                pass2_bboxes = [normalize_bbox(item['bbox_2d'], x_factor, y_factor) for item in pass2_data]
                pass2_labels = [item['label'] for item in pass2_data]
            else:
                pass2_bboxes = []
                pass2_labels = []

            # Parse Pass 3 input results (captioned bboxes to use for self critique)
            if pass3_input_json_str:
                pass3_input_data = json.loads(pass3_input_json_str)
                pass3_input_bboxes = [normalize_bbox(item['bbox_2d'], x_factor, y_factor) for item in pass3_input_data]
                pass3_input_labels = [item['label'] for item in pass3_input_data]
            else:
                pass3_input_bboxes = []
                pass3__input_labels = []
            
            # Create visualization
            fig, axes = plt.subplots(2, 3, figsize=(18, 6))
            fig.suptitle(f'Bbox Analysis for: {seg_name}', fontsize=16, fontweight='bold')
            
            # Plot 1: Pass 1 bboxes (initial detection)
            ax1 = axes[0, 0]
            ax1.imshow(image)
            ax1.set_title(f'Pass 1: Initial Detection\n({len(pass1_bboxes)} bboxes)')
            ax1.axis('off')
            
            for j, bbox in enumerate(pass1_bboxes):
                x1, y1, x2, y2 = bbox
                rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                       linewidth=2, edgecolor='red', facecolor='none')
                ax1.add_patch(rect)
                ax1.text(x1, y1-5, f'bbox {j+1}', fontsize=10, color='red', 
                        bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
            
            # Plot 2: Pass 2 bboxes with labels (same image, all bboxes)
            ax2 = axes[0, 1]
            ax2.imshow(image)
            ax2.set_title(f'Pass 2: Captioned Bboxes\n({len(pass2_bboxes)} bboxes)')
            ax2.axis('off')
            
            colors = ['blue', 'green', 'purple', 'orange', 'cyan']
            for j, (bbox, label) in enumerate(zip(pass2_bboxes, pass2_labels)):
                x1, y1, x2, y2 = bbox
                color = colors[j % len(colors)]
                rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                       linewidth=2, edgecolor=color, facecolor='none')
                ax2.add_patch(rect)
                ax2.text(x1, y1-5, f'{label}', fontsize=10, color=color, 
                        bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
            
            # Plot 3: Pass 3 input bboxes with labels (the output of pass 2 is filtered and used as input for pass 3)
            ax3 = axes[0, 2]
            ax3.imshow(image)

            ax3.set_title(f'Pass 3: Self-Critique Input Bboxes\n({len(pass3_input_bboxes)} bboxes)')
            ax3.axis('off') 
            for j, (bbox, label) in enumerate(zip(pass3_input_bboxes, pass3_input_labels)):
                x1, y1, x2, y2 = bbox
                color = colors[j % len(colors)]
                rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                       linewidth=2, edgecolor=color, facecolor='none', linestyle='--')
                ax3.add_patch(rect)
                ax3.text(x1, y1-5, f'{label}', fontsize=10, color=color, 
                        bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
            
            ax4 = axes[1, 0]
            ax4.imshow(image)
            ax4.set_title(f'Pass 3: Final BBox Prediction for \"{seg_name}\"')
            ax4.axis('off')
            for bbox in pass3_output_bbox:
                x1, y1, x2, y2 = bbox
                color = colors[j % len(colors)]
                rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                       linewidth=2, edgecolor=color, facecolor='none', linestyle='--')
                ax4.add_patch(rect)
            
            ax5 = axes[1, 1]
            ax5.imshow(image)

            predicted_overlay = np.zeros((pass3_predicted_mask.shape[0], pass3_predicted_mask.shape[1], 4))
            predicted_overlay[pass3_predicted_mask, 0] = 1.0  # Red color for prediction
            predicted_overlay[pass3_predicted_mask, 3] = 0.5  # Alpha transparency
            ax5.imshow(predicted_overlay)
            ax5.set_title(f'Predicted: {seg_name}')
            ax5.axis('off')

            ax6 = axes[1, 2]
            ax6.imshow(image)
            gt_mask = np.array(gt_mask)
            gt_overlay = np.zeros((gt_mask.shape[0], gt_mask.shape[1], 4))
            gt_overlay[gt_mask.astype(bool), 1] = 1.0  # Green color for ground truth
            gt_overlay[gt_mask.astype(bool), 3] = 0.5  # Alpha transparency
            ax6.imshow(gt_overlay)
            ax6.set_title(f'Ground Truth: {seg_name}')
            ax6.axis('off')

            # Add text box for thinking process
            fig.text(0.02, 0.02, pass3_output_think, fontsize=10, 
                     bbox=dict(boxstyle="round,pad=0.5", facecolor='lightgray', alpha=0.8),
                     verticalalignment='bottom')

                
            plt.tight_layout()
            # plt.savefig(f'bbox_analysis_{filename}_{i}_{seg_name.replace("/", "_").replace(" ", "_")}.png', 
            #            dpi=150, bbox_inches='tight')
            plt.show()

            

            # ax3.set_title('Comparison: Pass 1 (red) vs Pass 2 (colored)')
            # ax3.axis('off')
            
            # # Draw Pass 1 bboxes in red
            # for j, bbox in enumerate(pass1_bboxes):
            #     x1, y1, x2, y2 = bbox
            #     rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
            #                            linewidth=2, edgecolor='red', facecolor='none', 
            #                            linestyle='--', alpha=0.7)
            #     ax3.add_patch(rect)
            
            # # Draw Pass 2 bboxes in colors
            # for j, (bbox, label) in enumerate(zip(pass2_bboxes, pass2_labels)):
            #     x1, y1, x2, y2 = bbox
            #     color = colors[j % len(colors)]
            #     rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
            #                            linewidth=2, edgecolor=color, facecolor='none')
            #     ax3.add_patch(rect)


            
            # Additional individual bbox visualizations for Pass 2
            # if len(pass2_bboxes) > 1:
            #     fig2, axes2 = plt.subplots(1, len(pass2_bboxes), figsize=(6*len(pass2_bboxes), 6))
            #     if len(pass2_bboxes) == 1:
            #         axes2 = [axes2]
                    
            #     fig2.suptitle(f'Individual Pass 2 Bboxes for: {seg_name}', fontsize=16, fontweight='bold')
                
            #     for j, (bbox, label) in enumerate(zip(pass2_bboxes, pass2_labels)):
            #         ax = axes2[j]
            #         ax.imshow(image)
            #         ax.set_title(f'Bbox {j+1}: {label}')
            #         ax.axis('off')
                    
            #         x1, y1, x2, y2 = bbox
            #         color = colors[j % len(colors)]
            #         rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
            #                                linewidth=3, edgecolor=color, facecolor='none')
            #         ax.add_patch(rect)
                
            #     plt.tight_layout()
            #     # plt.savefig(f'individual_bboxes_{filename}_{i}_{seg_name.replace("/", "_").replace(" ", "_")}.png', 
            #     #            dpi=150, bbox_inches='tight')
            #     plt.show()
            
            # Print comparison information
            # print(f"\n=== Analysis for {seg_name} ===")
            # print(f"Pass 1 detected {len(pass1_bboxes)} bboxes")
            # print(f"Pass 2 captioned {len(pass2_bboxes)} bboxes")
            
            # if pass2_labels:
            #     print("Pass 2 Labels:")
            #     for j, label in enumerate(pass2_labels):
            #         print(f"  Bbox {j+1}: {label}")
                    
            #     # Check if labels match expected seg_name
            #     expected_lower = seg_name.lower()
            #     matching_labels = [label for label in pass2_labels if expected_lower in label.lower() or any(word in label.lower() for word in expected_lower.split())]
            #     print(f"Labels matching '{seg_name}': {len(matching_labels)}/{len(pass2_labels)}")
            
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON for {seg_name}: {e}")
        except Exception as e:
            print(f"Error processing {seg_name}: {e}")

In [83]:
## Code for getting pascalpart images and object/object parts 
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Models/GLAMM')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Datasets')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines')

from pascalpart import get_pascalpart_classes, get_pascalpart_masks
from utils import read_txt_file, save_to_json
from tqdm import tqdm
import os

pascal_image_dir = "/data/Pascal_VOC_2012/VOCdevkit/VOC2012/JPEGImages" # 17125 images
annotations_path= "/data/PartSegmentationDatasets/PascalPart/Annotations_Part"
val_filepath = "/data/PartSegmentationDatasets/PascalPart/val.txt" # 925 images. File contains just the file prefix. Add .jpg extension for images, and .mat extension for annotations
val_filenames = read_txt_file(val_filepath)

all_object_outputs = []
all_parts_outputs = []
for filename in tqdm(val_filenames[:10]):
    img_filepath = os.path.join(pascal_image_dir, filename+'.jpg')

    image = PILImage.open(img_filepath)
    image = image.convert("RGB")
    original_width, original_height = image.size
    resize_size = 840
    x_factor, y_factor = original_width/resize_size, original_height/resize_size

    annot_filename = filename + '.mat'
    anno_dict = get_pascalpart_masks(annot_filename, annotations_path, images_path=pascal_image_dir)
    
    # classes_to_detect = get_pascalpart_classes(annot_filename, annotations_path) # This is a dictionary in this format {object: [list of parts]}
    # print(classes_to_detect)
    for obj_name, anno in anno_dict.items():
        obj_masks = anno['object_maps']
        obj_mask = combine_masks(obj_masks)

        gt_mask_list = [obj_mask]
        seg_list = [obj_name]

        parts_masks = anno['parts']
        # print(parts_masks.keys())
        for part_name, masks in parts_masks.items():
            part_mask = combine_masks(masks)
            gt_mask_list.append(part_mask)
            part_full_name = obj_name + "'s " + part_name
            seg_list.append(part_full_name)

        # LLM inference needs to happen 3 times, once for the QUESTION_TEMPLATE, once for BOX_CAPTION_TEMPLATE, and once for SELF_CRITIQUE_TEMPLATE

        # break the following down into a method with required inputs 
        # we can create three different methods for each template if needed
        # each method will return its own output, which we can pass into the next method for the next template
        # This way, we dont need to loop over the templates, just call the methods in sequence
        # Begin creating prompt and running inference using the LLM 

        # A single batch is a single object and its parts. We have all these objects and parts in seg_list
        
        # Pass 1: QUESTION_TEMPLATE. This will give us the initial bounding boxes and points for the object and its parts
        # json_strings are list of strings. Each string corresponds to an object/part name in seg_list that we passed in. 
        # Each string is of the format: list of dicts. We have multiple dicts in the list if the object/part is present multiple times in the image
        # Each dict has keys "bbox_2d" and "point_2d" 
        pass1_json_strings, _, _, thinks = generate_outputs_for_question_template(
            filename,
            seg_list,
            image,
            resize_size=resize_size
        )

        # Pass 2: BOX_CAPTION_TEMPLATE. This will give us the caption for the first predicted region
        pass2_json_strings_cleaned = generate_outputs_for_caption_template(
            filename,
            pass1_json_strings,
            image,
            resize_size=resize_size
        )

        # print(pass2_json_strings_cleaned)

        # We should have the same number of outputs at this stage as the number of objects that we passed in (seg_list)

        assert len(pass1_json_strings) == len(seg_list)
        assert len(pass2_json_strings_cleaned) == len(seg_list)

        # pass2 can generate bboxes that were never present in pass1. 
        # This requires a filtering step to remove those bboxes from pass2 that were not in pass1.
        # Also, add the bboxes from pass1 that were not in pass2 to pass2 with a label of "unknown" or "unlabeled"

        # if the json strings are not valid jsons, we replace them with an empty list. 
        # This avoids running into JSONDecodeErrors while the code loops
        pass1_jsons = [safe_json_array_loads(js) for js in pass1_json_strings]
        pass2_jsons = [safe_json_array_loads(js) for js in pass2_json_strings_cleaned]

        # Filter pass2_jsons to only include bboxes that were in pass1_jsons
        filtered_pass2_jsons = []
        for p1, p2 in zip(pass1_jsons, pass2_jsons):
            # Recall: p1 and p2 are lists of dicts with keys "bbox_2d" and "point_2d"
            # Sometimes, p1 and p2 can have lists that are empty due to json parsing errors
            if all_inner_lists_empty(p1):
                # if all p1_bboxes is empty, all of p2 boxes should also be just empty. 
                filtered_pass2_jsons.append(p1)
                # We do not need to process further for this object/part
                continue

            p1_bboxes = [item['bbox_2d'] for item in p1]

            filtered_p2 = []
            for item in p2:
                item_bbox = item['bbox_2d']
                if item_bbox in p1_bboxes:
                    filtered_p2.append(item)
                    # remove from p1_bboxes to keep track of which bboxes have been matched
                    p1_bboxes.remove(item_bbox)
                else:
                    # if the bbox is not in p1_bboxes, we match it to the closest bbox in p1_bboxes based on bbox IoU. We only consider it a match if bbox IoU > 0.9
                    item_bbox_np = np.array(item_bbox)
                    best_iou = 0
                    best_match = None
                    
                    for p1_bbox in p1_bboxes:
                        p1_bbox_np = np.array(p1_bbox)
                        iou = compute_bbox_iou(item_bbox, p1_bbox)
                        if iou > best_iou:
                            best_iou = iou
                            best_match = p1_bbox

                    # consider it a match if IoU > 0.9
                    if best_iou > 0.9:
                        # Add the item with the matched bbox
                        item_copy = item.copy()
                        item_copy['bbox_2d'] = best_match
                        filtered_p2.append(item_copy)
                        p1_bboxes.remove(best_match)

                        
            # Add unmatched bboxes from p1 as "unlabeled"
            for unmatched_bbox in p1_bboxes:
                filtered_p2.append({
                    "bbox_2d": unmatched_bbox,
                    "label": "unlabeled"
                })

            filtered_pass2_jsons.append(filtered_p2)
        
        # Convert back to json strings
        # For the objects/parts where pass1 had no bboxes, we will have a list of empty lists in filtered_pass2_jsons. 
        ## These are also converted to corresponding strings, and passed to pass3 
        pass2_json_strings_filtered = [json.dumps(js) for js in filtered_pass2_jsons]


        # Pass 3: SELF_CRITIQUE_TEMPLATE. This will give us the refined bounding boxes and points for the object and its parts

        
        pass3_json_strings, pass3_bboxes, pass3_points, pass3_thinks = generate_outputs_for_self_critique_template(
            filename, 
            seg_list,
            pass2_json_strings_filtered, 
            image, 
            resize_size=840
        )

        pass3_predicted_masks = []
        with torch.inference_mode():
            for id_idx in range(len(pass3_bboxes)):
                if id_idx == 0:
                    all_outputs = all_object_outputs
                else:
                    all_outputs = all_parts_outputs

                think = pass3_thinks[id_idx]
                
                if len(pass3_bboxes[id_idx]) == 0:
                    intersection = 0
                    union = np.array(gt_mask_list[id_idx]).sum()
                    bbox_iou = 0.0
                    all_outputs.append({
                        "image_id": filename,
                        "ann_id": filename,
                        "seg_id": seg_list[id_idx],
                        "think": think,
                        "intersection": int(intersection),
                        "union": int(union),
                        "bbox_iou": bbox_iou,
                        "visualization_path": None
                    })
                    continue
                    
                try:
                    segmentation_model.set_image(image)
                    mask_all = np.zeros((original_height, original_width), dtype=bool)
                except Exception as e:
                    print("Set image error: ", e, filename)
                    # skip this because the image or mask is not correct
                    continue

                try:
                    bboxes = pass3_bboxes[id_idx]
                    points = pass3_points[id_idx]
                    for bbox, point in zip(bboxes, points):
                        masks, scores, _ = segmentation_model.predict(
                            point_coords=[point],
                            point_labels=[1],
                            box=bbox
                        )
                        sorted_ind = np.argsort(scores)[::-1]
                        masks = masks[sorted_ind]
                        mask = masks[0].astype(bool)
                        mask_all = np.logical_or(mask_all, mask)
                    gt_mask = np.array(gt_mask_list[id_idx])
                except Exception as e:
                    print("Segmentation error: ", e, filename)
                    # skip this because the image or mask is not correct
                    continue

                try:
                    intersection, union = compute_iou(mask_all, gt_mask)
                except Exception as e:
                    print("Image error: ", e)
                    # skip this because the image or mask is not correct
                    continue

                pass3_predicted_masks.append(mask_all)

                all_outputs.append({
                    "image_id": filename,
                    "ann_id": filename,
                    "seg_id": seg_list[id_idx],
                    "think": think,
                    "intersection": int(intersection),
                    "union": int(union),
                    "bbox_iou": 0,
                    "visualization_path": None
                })

        # print(f"pass3_json_strings: {pass3_json_strings}")
        # print(f"pass3_bboxes: {pass3_bboxes}")  
        # print(f"pass3_points: {pass3_points}")
        # print(f"pass3_thinks: {pass3_thinks}")

        ################ Visualization and analysis code ################
        # For Pass 2:
        ## We want to check if the captions generated match the object/part names that we passed. 
        ## We also want to see if the bounding boxes that we passed match the bounding boxes generated in the caption output 
        ## We also want to visualize the bounding boxes on the image to see if the generated labels/captions are correct.
        ## We want to visualize the filtered bboxes to make sure we are providig only the relevant bboxes to Pass 3 -- self critique and bbox refinement

        # For pass 3:
        ## We want to add one axis to the visualization that shows the predicted boxes from pass 3
        ## We want to add another axis that shows the predicted masks from pass 3
        ## We want to add another axis that shows the ground truth masks for comparison
        ## We want to add the pass3 thinking process as a text box on the visualization

        # visualize_bbox_comparison(image, pass1_json_strings, pass2_json_strings_cleaned, pass2_json_strings_filtered, pass3_bboxes, pass3_predicted_masks, gt_mask_list, pass3_thinks, seg_list, filename, x_factor, y_factor)

        # End of inference using the LLM        
        
        # break # doing it for one object and its parts for now

 30%|███       | 3/10 [04:06<09:18, 79.73s/it] 

Reasoning error:  'point_2d' Text:  <think> The previous answer correctly identified the headlight of the train. The captions "headlight" match the objects in the provided bboxes, which are indeed the headlights of the train. There is no need to adjust the bboxes as they are already correctly placed.</think>
<answer>[{"bbox_2d": [540, 341, 568, 366]}, {"bbox_2d": [647, 348, 666, 370]}]</answer> Image:  2009_003105 Seg:  train's headlight


100%|██████████| 10/10 [10:59<00:00, 65.91s/it]


In [84]:
def compute_giou(results):
    all_ious = []
        
    # process all items in each file
    for item in results:
        intersection = item['intersection']
        union = item['union']
        
        # calculate IoU of each item
        iou = intersection / union if union > 0 else 0
        all_ious.append({
            'image_id': item['image_id'],
            'iou': iou
        })
            
    # calculate gIoU
    gIoU = np.mean([item['iou'] for item in all_ious])
    # calculate cIoU

    # print the results
    print(f"gIoU (average of per image IoU): {gIoU:.4f}")

    return gIoU

In [88]:
compute_giou(all_object_outputs)

gIoU (average of per image IoU): 0.8472


np.float64(0.8471710200294912)